<a href="https://colab.research.google.com/github/vsidaarth/Transformer-Variants-for-HourvAhead-PV-Forecasting/blob/main/DLInear2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import plotly.express as px

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MovingAvg(nn.Module):
    def __init__(self, kernel_size=25):
        super().__init__()
        self.kernel_size = kernel_size

    def forward(self, x):  # x: (B, L)
        k = min(self.kernel_size, x.size(1))
        w = torch.ones(1, 1, k, device=x.device) / k
        pad = k - 1
        x_pad = F.pad(x.unsqueeze(1), (pad//2, pad - pad//2), mode='replicate')
        y = F.conv1d(x_pad, w).squeeze(1)
        return y[:, :x.size(1)]  # trim


class DLinear(nn.Module):
    """
    Decompose per-channel series into trend + seasonal, then linear maps from seq_len -> pred_len.
    Input:  (B, L, C)
    Output: (B, pred_len, C)
    """
    def __init__(self, seq_len, pred_len, enc_in, individual=True, moving_avg=25):
        super().__init__()
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in
        self.individual = individual
        self.ma = MovingAvg(moving_avg)

        if individual:
            self.lin_trend  = nn.ModuleList([nn.Linear(seq_len, pred_len) for _ in range(enc_in)])
            self.lin_season = nn.ModuleList([nn.Linear(seq_len, pred_len) for _ in range(enc_in)])
        else:
            self.lin_trend  = nn.Linear(seq_len, pred_len)
            self.lin_season = nn.Linear(seq_len, pred_len)

    def forward(self, x):  # (B, L, C)
        B, L, C = x.shape
        x_tc = x.permute(0, 2, 1).contiguous()    # (B, C, L)
        trend   = torch.stack([self.ma(x_tc[:, c, :]) for c in range(C)], dim=1)   # (B, C, L)
        seasonal = x_tc - trend
        if self.individual:
            out_t = [self.lin_trend[c](trend[:, c, :])   for c in range(C)]  # list of (B, pred_len)
            out_s = [self.lin_season[c](seasonal[:, c, :]) for c in range(C)]
            out   = torch.stack([ot + os for ot, os in zip(out_t, out_s)], dim=2)  # (B, pred_len, C)
        else:
            out_t = self.lin_trend(trend)      # (B, C, pred_len)
            out_s = self.lin_season(seasonal)  # (B, C, pred_len)
            out   = (out_t + out_s).permute(0, 2, 1).contiguous()  # (B, pred_len, C)
        return out


In [ ]:
# load the data
df = pd.read_csv('/content/updated_timetable.txt')
df.columns=['datetime','irradiation_forecast','temperature_forecast','irradiation','temperature','power']
tdi = pd.to_datetime(df['datetime'], format='%d-%m-%y %H')
df.set_index(tdi, inplace=True)

# # Define date ranges to remove
# ranges_to_remove = [
#     ('2018-05-01', '2018-05-22'),
#     ('2019-09-09', '2019-09-28'),
#     ('2019-11-29', '2020-01-20'),
#     ('2013-07-23', '2013-08-26')
# ]

# # Create a boolean mask for rows to keep
# mask = pd.Series(True, index=df.index)
# for start, end in ranges_to_remove:
#     mask &= ~((df.index >= start) & (df.index <= end))

# # Apply the mask to the DataFrame
# data = df[mask]
start_date = '2014-01-01'
end_date = '2019-01-31'
# Filter the data for the specified date range
data = df[start_date:end_date]

data = data.drop(columns=['datetime','irradiation_forecast','temperature_forecast'])
data['power'] = data['power']/1000

# data['power'] = data['power']/data['power'].max()*317
# data = data[59905::]

In [ ]:
load = data.loc[:, 'power']
scaler = MinMaxScaler()
load = scaler.fit_transform(load.values.reshape(-1, 1)).flatten()

irradiation = data.loc[:, 'irradiation']
scaler_w = MinMaxScaler()
irradiation = scaler_w.fit_transform(irradiation.values.reshape(-1, 1)).flatten()

load_scaled = pd.DataFrame({
    'power_generation': load,
    'irradiation': irradiation
}, index=data.index)


shift_1 = load_scaled.shift(1,axis = 0)
shift_1.columns = ['power_generation(t-1)', 'irradiation(t-1)']

final_table = pd.concat([load_scaled['power_generation'],shift_1],axis = 1)
final_table = final_table.dropna(axis=0)

# Extracting components
final_table['hour'] = final_table.index.hour
final_table['day_of_week'] = final_table.index.weekday
final_table['month'] = final_table.index.month

# Applying cyclical encoding
final_table['hour_sin'] = np.sin(2 * np.pi * final_table['hour'] / 24)
final_table['hour_cos'] = np.cos(2 * np.pi * final_table['hour'] / 24)
final_table['month_sin'] = np.sin(2 * np.pi * final_table['month'] / 12)
final_table['month_cos'] = np.cos(2 * np.pi * final_table['month'] / 12)

# Optionally drop the original time components if they are no longer needed
final_table.drop(['hour', 'day_of_week', 'month'], axis=1, inplace=True)

# Show the modified DataFrame
final_table.head(-1)


,power_generation,power_generation(t-1),irradiation(t-1),hour_sin,hour_cos,month_sin,month_cos
datetime,,,,,,,
2014-01-01 01:00:00,0.0,0.0,0.000441,0.258819,9.659258e-01,0.5,0.866025
2014-01-01 02:00:00,0.0,0.0,0.000336,0.500000,8.660254e-01,0.5,0.866025
2014-01-01 03:00:00,0.0,0.0,0.000228,0.707107,7.071068e-01,0.5,0.866025
2014-01-01 04:00:00,0.0,0.0,0.000186,0.866025,5.000000e-01,0.5,0.866025
2014-01-01 05:00:00,0.0,0.0,0.000017,0.965926,2.588190e-01,0.5,0.866025
...,...,...,...,...,...,...,...
2019-01-31 18:00:00,0.0,0.0,0.000892,-1.000000,-1.836970e-16,0.5,0.866025
2019-01-31 19:00:00,0.0,0.0,0.000920,-0.965926,2.588190e-01,0.5,0.866025
2019-01-31 20:00:00,0.0,0.0,0.001194,-0.866025,5.000000e-01,0.5,0.866025


In [ ]:
test_set_start_date = final_table.index[-1] - pd.DateOffset(months=12)
train_set = final_table[final_table.index < test_set_start_date]
test_set = final_table[final_table.index >= test_set_start_date]

In [ ]:
def split_sequences_autoformer(input_par, output, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(input_par)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1
        if out_end_ix >= len(input_par):
            break
        seq_x = input_par[i:end_ix, :]
        seq_y = output[out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
NEW_SEQ_LEN  = 72
NEW_LABEL_LEN = 36
n_features = 6

n_steps_in = NEW_SEQ_LEN # new seq_len
n_steps_out = 1  # forecast horizon
input_sp = train_set.iloc[:,1:7]
output_sp = pd.DataFrame(train_set.loc[:,'power_generation'])
# Split sequences again
X_auto, y_auto = split_sequences_autoformer(input_sp.values, output_sp.values, n_steps_in, n_steps_out)

# Train-test split as before
input_train = train_set.iloc[:, 1:8]
output_train = pd.DataFrame(train_set['power_generation'])
input_test = test_set.iloc[:, 1:8]
output_test = pd.DataFrame(test_set['power_generation'])

X_train_auto, y_train_auto = split_sequences_autoformer(input_train.values, output_train.values, n_steps_in, n_steps_out)
X_test_auto, y_test_auto = split_sequences_autoformer(input_test.values, output_test.values, n_steps_in, n_steps_out)

# Convert again to torch tensors
X_train_tensor = torch.tensor(X_train_auto, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_auto, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_auto, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_auto, dtype=torch.float32)
tgt_ch = 0  # index of 'power_generation' inside your feature tensor (adjust if needed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = DLinear(
    seq_len=NEW_SEQ_LEN,     # 48 in your example
    pred_len=1,              # t+1
    enc_in=n_features,       # 6 in your example
    individual=True,
    moving_avg=50            # try 25 and 49
).to(device)

loss_fn  = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
EPOCHS = 10   # fast to train; bump a bit


In [ ]:
from torch.utils.data import DataLoader, Dataset

class LoadDataset(Dataset):
    def __init__(self, X, y):
        self.X = X; self.y = y
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

train_dataset = LoadDataset(X_train_tensor, y_train_tensor)
train_loader  = DataLoader(train_dataset, batch_size=64, shuffle=True)

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)                # (B, L, C)
        batch_y = batch_y.to(device).view(-1)       # (B,)
        pred = model(batch_X)[:, 0, tgt_ch]         # (B, 1, C) -> (B,)
        loss = loss_fn(pred, batch_y)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        epoch_loss += loss.item()
    print(f"Epoch {epoch+1:02d} | loss {epoch_loss/len(train_loader):.5f}")


Epoch 01 | loss 0.01446
Epoch 02 | loss 0.01144
Epoch 03 | loss 0.01125
Epoch 04 | loss 0.01120
Epoch 05 | loss 0.01121
Epoch 06 | loss 0.01117
Epoch 07 | loss 0.01114
Epoch 08 | loss 0.01111
Epoch 09 | loss 0.01114
Epoch 10 | loss 0.01113


In [ ]:
model.eval()
preds = []
with torch.no_grad():
    for i in range(0, len(X_test_tensor), 64):
        batch_X = X_test_tensor[i:i+64].to(device)
        p = model(batch_X)[:, 0, tgt_ch].detach().cpu().numpy()
        preds.append(p)

preds = np.concatenate(preds, axis=0).reshape(-1,1)   # (N_test,1)
truth = y_test_tensor.cpu().numpy().reshape(-1,1)

# If your MinMax scaler 'scaler' was fit on the target column only:
preds_orig1 = scaler.inverse_transform(preds).ravel()
truth_orig1 = scaler.inverse_transform(truth).ravel()

from sklearn.metrics import mean_absolute_error, mean_squared_error

mae  = mean_absolute_error(truth_orig1, preds_orig1)
mse  = mean_squared_error(truth_orig1, preds_orig1)   # <- compatible everywhere
rmse = np.sqrt(mse)
mbe  = (preds_orig1 - truth_orig1).mean()
print(f"MAE: {mae:.3f} | RMSE: {rmse:.3f} | MBE: {mbe:.3f}")


MAE: 0.251 | RMSE: 0.461 | MBE: 0.003


In [ ]:
import os
import pandas as pd
import numpy as np

# === Function to save forecasts ===
def save_forecasts(truth_orig, preds_orig, test_set, model_name, filename):
    # Ensure save folder exists in Google Drive
    save_dir = "/content/drive/MyDrive/Colab Notebooks/Forecasts"
    os.makedirs(save_dir, exist_ok=True)

    # Build full output path
    out_path = os.path.join(save_dir, filename)

    # Align lengths
    n = min(len(truth_orig), len(preds_orig), len(test_set.index))

    # Build dataframe
    df = pd.DataFrame({
        "datetime": pd.to_datetime(test_set.index[:n]),
        "actual": np.asarray(truth_orig).reshape(-1)[:n],
        "forecast": np.asarray(preds_orig).reshape(-1)[:n],
        "model": model_name
    })

    # Save to CSV
    df.to_csv(out_path, index=False)
    print(f"[{model_name}] CSV saved to: {out_path}")
    return df.head()

# === Example: Save Informer results ===
save_forecasts(
    truth_orig1,      # your ground truth (kW)
    preds_orig1,      # your model forecasts (kW)
    test_set,                  # DataFrame with datetime index
    "DLinear",                # model name
    "preds_linear.csv"       # file name in Drive
)


[DLinear] CSV saved to: /content/drive/MyDrive/Colab Notebooks/Forecasts/preds_linear.csv


,datetime,actual,forecast,model
0,2018-01-31 23:00:00,0.0,0.059943,DLinear
1,2018-02-01 00:00:00,0.0,0.064210,DLinear
2,2018-02-01 01:00:00,0.0,0.053655,DLinear
3,2018-02-01 02:00:00,0.0,0.019397,DLinear
4,2018-02-01 03:00:00,0.0,0.003123,DLinear
